# Problem Set 4

Find a dataset that is currently in a table that you think could be turned into a multi-table database. This can be any kind of data that interests you.

1) Describe the data. Write a metadata document that describes the source of the data, what the data are, what each column is, etc. (4 pts.)

 **Please see "contact_info.md" for metadata**

2) Normalize the data. Write descriptions (words, drawings, or tables are fine--not code at this point) of the tables you could use to normalize these data. Describe why you chose the normalize the data in this manner. Include the column names, data types for each column, primary and foreign keys you would use. (8pts.)

**Normalize the data is good for downstring analyses. For examples, some time your data are generated from different source, and potneially these data could have different ways of rounding, different units, or empty vulaes in different cells.**

**I ususally try to make sure each column has its own data type, and remove the missing values.**

**I'm going to create two tables**

 The first table called "Contact_phone", including ID (as primary keys), Name, Phone_No, and Fax_No, and data types are integer, string, string and string, respectively. I'll use "Name" as foreign keys
  The second table called "City", including ID (as primary keys), Name, and City; the data types are integer, string, and string, respectively. I'll use "ID" as primary keys, and "Name" as foreign keys

 
3) Either directly in sqlite or in Python with SQLAlchemy, write the code needed to define the tables above. (6pts.)

 see below

4) Write the code to load the data into the database. (2pts) Note this is only 2 points! Don't spend a ton of time on this. If it turns into a headache, don't worry about it.

  see below

Extra Credit: Find another dataset, briefly describe the dataset and columns. Briefly describe the tables you would use and how you would normalize these data. (2pts.)
    
# Data used is "Contact_Info.csv"

# Create database with 2 tables

In [55]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import DateTime, Boolean
from sqlalchemy import exists
from sqlalchemy import sql, select, join, desc


# Create a sqlite database 
engine = create_engine('sqlite:///Contact_Info.sqlite')

metadata=MetaData(engine)

# Try to load Contact_phone info from database, if not there, create it.
try:
    Contact_phone=Table('Contact_phone', metadata, autoload=True)
except:
    Contact_phone = Table ('Contact_phone', metadata,
                Column('ID', Integer, autoincrement=True, primary_key=True),
                Column('Name', String),
                Column('Phone_No', String),
                Column('Fax_No', String)
               )

# Same for City table.
try:
    City=Table('City', metadata, autoload=True)
except:
    City = Table ('City', metadata,
                 #Column('ID', Integer, autoincrement=True, primary_key=True),
                 Column('Name', String, ForeignKey('Contact_phone.Name')),
                 Column('City', String),
                )
                 
                 
metadata.create_all(engine)

# Add data to sqlite database

In [56]:
import csv

info = open("Contact_Info.csv")


reader = csv.DictReader(info)

City_dict={}

# Read through the file and make a dictionary for Name.
# This gets a unique list of names.
for Line in reader:
    #print(Line)
    if Line['Name'] not in City_dict:
        City_dict[Line['Name']]=[Line['Name'], Line['City']]

#print(City_dict)
for key, value in City_dict.items(): 
    print(value[1])

Pittsburgh
Baltimore
Houston
Suide
NA
Gainesville


In [57]:
# Add the City_dict and people's name to the City table

conn = engine.connect()


def insert_city(name, city):
    ins=City.insert().values(Name=name,
                                 City=city)
    
    result = conn.execute(ins)

for key, value in City_dict.items(): 
    insert_city(key, value[1])


In [58]:
print(City_dict)

{'Joel Garner': ['Joel Garner', 'Pittsburgh'], 'Clive Lloyd': ['Clive Lloyd', 'Baltimore'], 'Gordon Greenidge': ['Gordon Greenidge', 'Houston'], 'NA': ['NA', 'Suide'], 'Alex Sun': ['Alex Sun', 'NA'], 'Ziyu Fu': ['Ziyu Fu', 'Gainesville']}


In [59]:
query=select([City.c.City])
             
result = conn.execute(query)
for row in result:
    print(row)

('Pittsburgh',)
('Baltimore',)
('Houston',)
('Suide',)
('NA',)
('Gainesville',)


In [60]:
import pandas as pd

info.close()
info=open("Contact_Info.csv")
reader = csv.DictReader(info)
for Line in reader:
    try:
        Name = str(Line['Name'])
    except:
        skip
    ins=Contact_phone.insert().values(ID=Line['ID'],
                                Name = Line['Name'],
                                Phone_No = Line['Phone_No'],
                                Fax_No = Line['Fax_No'],
                                )
    result = conn.execute(ins)
    

In [61]:
from sqlalchemy import select

query=select([Contact_phone.c.Name, Contact_phone.c.Phone_No])
             
result = conn.execute(query)
for row in result:
    print(row)

('Joel Garner', '412-212-5421')
('Alex Sun', '352-328-0945')
('NA', '352-284-0928')
('Clive Lloyd', '410-306-1420')
('Gordon Greenidge', '281-564-6720')
('Ziyu Fu', 'NA')


# Join
jion the two table that only containing the infomation that person's name, phone number, fax number, and which city he live in.

In [63]:
q = select([City.c.City, Contact_phone.c.Phone_No, Contact_phone.c.Fax_No]).where(City.c.Name==Contact_phone.c.Name)


In [64]:
result3 = conn.execute(q)
for row in result3:
    print(row)

('Pittsburgh', '412-212-5421', '412-212-5400')
('Baltimore', '410-306-1420', '410-306-5400')
('Houston', '281-564-6720', '281-511-6600')
('Suide', '352-284-0928', '410-306-1420')
('NA', '352-328-0945', '281-564-6720')
('Gainesville', 'NA', 'xx-777-8830')
